In [1]:
import pandas as pd

In [8]:
names = ["prev", "curr", "type", "n"]
de_wiki = pd.read_csv("../data/22/clickstream-dewiki-2022-03.tsv", sep="\t", names=names)

In [9]:
de_wiki.head(5)

,prev,curr,type,n
0,other-empty,Lichtröhre,external,53.0
1,Lichtleiter,Lichtröhre,link,12.0
2,Wohnhöhle,Lichtröhre,link,19.0
3,other-search,Lichtröhre,external,76.0
4,other-internal,Lichtröhre,external,11.0
